

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

-Anthony Olcek

-N18364039

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
# @title
# %%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-f6cwdm_2/unsloth_3f559ae0df734cd2a080e4dfa989294d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-f6cwdm_2/unsloth_3f559ae0df734cd2a080e4dfa989294d
  Resolved https://github.com/unslothai/unsloth.git to commit d707bd43b4e883b521761d525be2fae428fe5980
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.10.12-py3-none-any.whl size=351539 sha256=c862d7b0227426b296cdd43d6c224747a1be8cbffea9cb7ea3b07ef1ae8b8f18
  Stored in directory: /tmp/pip-ephem-wheel-cache-992wbe7y/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.10.8
    Uninstalling unsloth-2025.10.8:
      Successfully uninstalled unsloth-2025.10.8

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@bbdab300de3eb76a435999e92815de452560e51d"

  Using cached unsloth-2025.10.8-py3-none-any.whl
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.10.12
    Uninstalling unsloth-2025.10.12:
      Successfully uninstalled unsloth-2025.10.12


## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1032  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

shuffled = full_dataset.shuffle(seed=42)

train_size = 5000
val_size = 500

train_dataset = shuffled.select(range(train_size))
validation_dataset = shuffled.select(range(train_size, train_size + val_size))

# # Shuffle the dataset for randomness and create our smaller splits
# shuffled_dataset = full_dataset.shuffle(seed=129)
# train_dataset = shuffled_dataset.select(range(5000))      # Use the first 5,000 for training
# validation_dataset = shuffled_dataset.select(range(5000, 5500)) # Use the next 500 for validation

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_true = sum(train_dataset["is_correct"])
train_false = len(train_dataset) - train_true
print(f"\n📊 Training set: {train_true} True, {train_false} False")
print(f"   Balance: {train_true/len(train_dataset):.1%}")

val_true = sum(validation_dataset["is_correct"])
val_false = len(validation_dataset) - val_true
print(f"📊 Validation set: {val_true} True, {val_false} False")
print(f"   Balance: {val_true/len(validation_dataset):.1%}\n")



📊 Training set: 1955 True, 3045 False
   Balance: 39.1%
📊 Validation set: 193 True, 307 False
   Balance: 38.6%



In [ ]:
# The instructional prompt template for training
# training_prompt = """You are a great mathematician. You are tasked to determine if the final answer in the SOLUTION is correct based on the problem described in the QUESTION. The SOLUTION may contain textual explanations, LaTeX formulas, and Python code blocks.
# Respond with only a single word: 'True' or 'False'.
# Below is the Question and Solution.
# Question:
# {}
# Solution:
# {}
# Output:
# {}"""

training_prompt = """You are a great mathematician. You are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 48, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 96, # A common practice is to set alpha = 2 * r
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Already have LoRA adapters! We shall skip this step.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
import math
import numpy as np
import torch
from sklearn.model_selection import KFold
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

# ---- config ----
N_FOLDS = 5
SEED = 42
OUTPUT_DIR = "outputs_kfold"
MAX_EPOCHS = 40
PATIENCE = 1
LR = 8e-5

torch.backends.cudnn.benchmark = True
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
indices = np.arange(len(formatted_train_dataset))
fold_losses = []

# detect eval arg name once
try:
    _ = TrainingArguments(output_dir="tmp", evaluation_strategy="epoch")
    EVAL_ARG = "evaluation_strategy"
except TypeError:
    EVAL_ARG = "eval_strategy"

base_args = dict(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=LR,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    # we'll set num_train_epochs=1 for per-call training and loop outside
    num_train_epochs=1,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    weight_decay=0.01,
    max_grad_norm=0.3,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    logging_steps=100,
    gradient_checkpointing=True,
    dataloader_num_workers=2,
    remove_unused_columns=True,
    seed=SEED,
    report_to="none",
    disable_tqdm=False,  # allow per-epoch progress bars
)

for fold, (train_idx, val_idx) in enumerate(kf.split(indices), start=1):
    print(f"\n=== Fold {fold}/{N_FOLDS} ===")
    train_ds = formatted_train_dataset.select(train_idx.tolist())
    val_ds = formatted_train_dataset.select(val_idx.tolist())

    args = {**base_args, "output_dir": f"{OUTPUT_DIR}/fold_{fold}"}
    args[EVAL_ARG] = "epoch"

    # compute steps per epoch for display/clarity
    world_size = max(1, torch.cuda.device_count())
    per_device = args["per_device_train_batch_size"]
    grad_acc = args["gradient_accumulation_steps"]
    batch_effective = per_device * grad_acc * world_size
    steps_per_epoch = math.ceil(len(train_ds) / batch_effective)
    total_steps = steps_per_epoch * MAX_EPOCHS
    print(f"Steps/epoch: {steps_per_epoch}, Total steps (approx): {total_steps}")

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,
        args=TrainingArguments(**args),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
    )

    # train one epoch per loop so progress shows per-epoch steps
    fold_val_losses = []
    for epoch in range(1, MAX_EPOCHS + 1):
        print(f"\n--- Fold {fold} epoch {epoch}/{MAX_EPOCHS} ---")
        trainer.train()               # trains for 1 epoch (progress will be step/steps_per_epoch)
        eval_metrics = trainer.evaluate()
        val_loss = eval_metrics.get("eval_loss", None)
        print(f"Epoch {epoch} val_loss = {val_loss:.4f}" if val_loss is not None else "No eval_loss returned")
        fold_val_losses.append(val_loss)

        # check if early stopping triggered by callback
        # trainer.control.should_training_stop is set by EarlyStoppingCallback
        if getattr(trainer, "control", None) and getattr(trainer.control, "should_training_stop", False):
            print(f"Early stopping triggered at epoch {epoch}")
            break

    # choose last eval loss (or best if you prefer to track min)
    chosen_loss = next((l for l in reversed(fold_val_losses) if l is not None), None)
    print(f"Fold {fold} chosen eval_loss = {chosen_loss:.4f}" if chosen_loss is not None else "Fold had no eval_loss")
    fold_losses.append(chosen_loss)

mean_loss = float(np.mean([l for l in fold_losses if l is not None]))
std_loss = float(np.std([l for l in fold_losses if l is not None]))
print(f"\nK-Fold Summary: mean eval_loss = {mean_loss:.4f} ± {std_loss:.4f}")




=== Fold 1/5 ===
Steps/epoch: 125, Total steps (approx): 5000

--- Fold 1 epoch 1/40 ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 3,407,872 of 7,245,139,968 (0.05% trained)


Epoch,Training Loss,Validation Loss
1,0.747200,0.620499


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Epoch 1 val_loss = 0.6205
Early stopping triggered at epoch 1
Fold 1 chosen eval_loss = 0.6205

=== Fold 2/5 ===
Steps/epoch: 125, Total steps (approx): 5000


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/4000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/1000 [00:00<?, ? examples/s]


--- Fold 2 epoch 1/40 ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 3,407,872 of 7,245,139,968 (0.05% trained)


ValueError: Attempting to unscale FP16 gradients.

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
# trainer.train()

# Save whatever is in memory now (fold 1)
model.save_pretrained("outputs_kfold/fold_1/interrupted")
tokenizer.save_pretrained("outputs_kfold/fold_1/interrupted")


In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch

DEVICE = next(model.parameters()).device
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

MAX_NEW_TOKENS = 8
ctx = getattr(model.config, "max_position_embeddings", 2048)
MAX_INPUT_LEN = max(256, ctx - MAX_NEW_TOKENS - 16)

model.eval()

inference_prompt = """You are a great mathematician. You are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    return 'true' in output_part.lower()

val_predictions, true_labels = [], []

for example in tqdm(validation_dataset, desc="Evaluating on validation set"):
    question = example["question"]
    solution = example["solution"]
    true_labels.append(example["is_correct"])

    prompt = inference_prompt.format(question, str(solution))
    enc = tokenizer([prompt], return_tensors="pt",
                    truncation=True, max_length=MAX_INPUT_LEN,
                    padding=False).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **enc,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=True,
        )

    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    val_predictions.append(parse_output(response_text))

accuracy = accuracy_score(true_labels, val_predictions)
print(f"\nValidation Accuracy: {accuracy:.4f}")

print("\n--- Sample Validation Results (Prediction vs. True Label) ---")
for i in range(min(10, len(val_predictions))):
    print(f"Sample {i+1}: Prediction={val_predictions[i]}, True Label={true_labels[i]}")



## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

xxx


In [ ]:
# Finish K-Fold with only 60 steps per remaining fold (2..N)
from copy import deepcopy
import numpy as np
from sklearn.model_selection import KFold
from trl import SFTTrainer
from transformers import TrainingArguments

# --- Same CV split as before ---
N_FOLDS = 5
SEED = 42
OUTPUT_DIR = "outputs_kfold"
LR = 8e-5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
indices = np.arange(len(formatted_train_dataset))

# Detect correct evaluation arg name (transformers 4.x vs 5.x)
try:
    _ = TrainingArguments(output_dir="tmp", evaluation_strategy="epoch")
    EVAL_ARG = "evaluation_strategy"
except TypeError:
    EVAL_ARG = "eval_strategy"

# If you already trained fold 1, start from fold 2:
START_FOLD = 2

fold_losses = []

for fold, (train_idx, val_idx) in enumerate(kf.split(indices), 1):
    if fold < START_FOLD:
        continue
    print(f"\n========== Fold {fold}/{N_FOLDS} (60 steps) ==========")

    train_split = formatted_train_dataset.select(train_idx.tolist())
    val_split   = formatted_train_dataset.select(val_idx.tolist())

    # Avoid carryover: clone the current base model for this fold
    model_fold = deepcopy(model)

    args_dict = {
        "output_dir": f"{OUTPUT_DIR}/fold_{fold}",
        "per_device_train_batch_size": 4,
        "gradient_accumulation_steps": 8,
        "learning_rate": LR,
        "warmup_ratio": 0.05,
        "lr_scheduler_type": "cosine",
        "num_train_epochs": 100,     # won't be reached; max_steps caps the run
        "max_steps": 60,             # <-- limit each fold to 60 update steps
        "fp16": not torch.cuda.is_bf16_supported(),
        "bf16": torch.cuda.is_bf16_supported(),
        "weight_decay": 0.01,
        "max_grad_norm": 0.3,
        "save_strategy": "steps",    # save at step granularity
        "save_steps": 60,            # save once at the end of 60 steps
        "save_total_limit": 1,
        "load_best_model_at_end": True,
        "logging_steps": 10,
        "gradient_checkpointing": True,
        "dataloader_num_workers": 2,
        "remove_unused_columns": True,
        "seed": SEED,
        "report_to": "none",
    }
    args_dict[EVAL_ARG] = "steps"
    args_dict["eval_steps"] = 60      # evaluate at the end of 60 steps

    trainer = SFTTrainer(
        model=model_fold,
        tokenizer=tokenizer,
        train_dataset=train_split,
        eval_dataset=val_split,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,
        args=TrainingArguments(**args_dict),
        # Early stopping is unnecessary with max_steps=60, so omitted
    )

    trainer.train()
    metrics = trainer.evaluate()
    val_loss = metrics.get("eval_loss", None)
    print(f"Fold {fold} eval_loss = {val_loss:.4f}")
    fold_losses.append(val_loss)

# Summary over the short folds
if fold_losses:
    import numpy as np
    print(f"\nShort-run K-Fold Summary (60 steps/fold): "
          f"mean eval_loss = {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
